In [ ]:
#%matplotlib widget

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
from neural_network.cam.cams import SmoothGradCAMpp, SSCAM, ScoreCAM
from neural_network.trainer import Agent
from neural_network.utils.utils import interactive_slices,interactive_slices_masked
from skimage.transform import resize

In [4]:
agent = Agent('resnet50');

Setup configurations...
Dataset sizes - Training: 421 Validation: 47 Test: 0
architecture [ResNet] was created
model [LightningModel] was created


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [5]:
agent.fit()

initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 46.2 M
---------------------------------
46.2 M    Trainable params
0         Non-trainable params
46.2 M    Total params
184.645   Total estimated model params size (MB)



Epoch 1 [71/71] {'loss': '1.16'}
Epoch 2 [71/71] {'loss': '1.14'}
Epoch 3 [71/71] {'loss': '1.07'}}
Epoch 4 [71/71] {'loss': '1.21'}}
Epoch 5 [71/71] {'loss': '1.02'}}
Epoch 6 [71/71] {'loss': '0.973'}
Epoch 7 [71/71] {'loss': '0.856'}
Epoch 8 [71/71] {'loss': '0.826'}
Epoch 9 [71/71] {'loss': '0.837'}
Epoch 10 [71/71] {'loss': '0.792'}
Epoch 11 [71/71] {'loss': '0.803'}
Epoch 12 [71/71] {'loss': '0.648'}
Epoch 13 [71/71] {'loss': '0.861'}
Epoch 14 [71/71] {'loss': '0.687'}
Epoch 15 [71/71] {'loss': '0.792'}
Epoch 16 [71/71] {'loss': '0.577'}
Epoch 17 [71/71] {'loss': '0.579'}
Epoch 18 [71/71] {'loss': '0.632'}
Epoch 19 [71/71] {'loss': '0.602'}
Epoch 20 [71/71] {'loss': '0.572'}
Epoch 21 [71/71] {'loss': '0.543'}
Epoch 22 [71/71] {'loss': '0.537'}
Epoch 23 [71/71] {'loss': '0.622'}
Epoch 24 [71/71] {'loss': '0.519'}
Epoch 25 [71/71] {'loss': '0.263'}
Epoch 26 [71/71] {'loss': '0.223'}
Epoch 27 [71/71] {'loss': '0.262'}
Epoch 28 [71/71] {'loss': '0.304'}
Epoch 29 [71/71] {'loss': '0.3

/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Traceback (most recent call last):
  File "/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/multiprocessing/queues.py", line 251, in _feed
    send_bytes(obj)
  File "/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/multiprocessing/connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/multiprocessing/connection.py", line 416, in _send_bytes
    self._send(header + buf)
  File "/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/multiprocessing/connection.py", line 373, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


Profiler Report

Action                             	| 

In [ ]:
agent.model;

In [90]:
val_iter = iter(agent.dataset.val_dataloader())

In [ ]:
first_image, first_label = next(val_iter)

In [ ]:
# Import M3d-CAM
from medcam import medcam

# Init your model and dataloader
model = agent.model
data_loader = agent.dataset.val_dataloader()

# Inject model with M3d-CAM
model = medcam.inject(model, backend='gcampp',output_dir="attention_maps", label=0,layer='model.layer4',save_maps=True,data_shape=(79,224,224))

model.eval()
output = model(first_image[0].unsqueeze(0))

In [39]:
darp = medcam.medcam_utils.normalize(model.medcam_dict['current_attention_map'])
darp = resize(darp.squeeze(0).squeeze(0),(79,224,224))

In [112]:
%matplotlib widget

In [114]:
model.eval()
plt.close()
#for first_image, first_label in agent.dataset.val_dataloader():

output = model(first_image[0].unsqueeze(0))

asd = interactive_slices_masked()
asd.multi_slice_viewer(first_image[0],darp)
#asd.cycle(0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyboardInterrupt: 

In [69]:
import numpy as np
import matplotlib.pyplot as plt
fig = plt.figure()
plt.imshow((first_image[0].squeeze(0)[50] * 255).numpy().astype(np.uint8), cmap='gray')
plt.imshow((darp[50] * 255).astype(np.uint8), cmap='jet', alpha=0.3)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
from neural_network.utils.utils import interactive_slices
testplot = interactive_slices()
testplot.multi_slice_viewer(darp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
from neural_network.cam.cam import SmoothGradCAMpp

In [64]:
wrapped_model = SmoothGradCAMpp(model,target_layer=model.model.layer4[1].conv2)

In [65]:
first_image[0].unsqueeze(0).shape

torch.Size([1, 1, 79, 224, 224])

In [68]:
model.eval()
cam, idx = wrapped_model(first_image[0].unsqueeze(0))

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.